In [1]:
import faiss
print("FAISS OK!", faiss.__version__)


ModuleNotFoundError: No module named 'faiss'

In [2]:
!pip install sentence-transformers faiss-cpu pandas langdetect indic-transliteration emoji scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.6/159.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 36.2 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=5ec437ae9e98a4aae1604a149303ce29e9bd824ca7bc3f9d50d772b8dda07586
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [3]:
!pip install sentence-transformers faiss-cpu pandas langdetect indic-transliteration emoji scikit-learn indic-nlp-library

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/factory.py", line 169, in _make_candidate_from_dist
    base = self._installed_candidate_cache[dist.canonical_name]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
KeyError: 'numpy'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/u

In [ ]:
!pip install sentence-transformers faiss-cpu pandas emoji scikit-learn
!pip install langdetect
!pip install indic-transliteration
!pip install indic-nlp-library
!pip install pytorch

In [ ]:
!pip install sentence-transformers faiss-cpu pandas langdetect indic-transliteration emoji scikit-learn indic-nlp-library transformers

In [ ]:
# ==============================================================
# BlueRadar – Multilingual Hazard Vector DB (Create / Update)
# Version 2.0 - Enhanced with better coverage and validation
# Supports: English + Hindi + Marathi + Hinglish
# ==============================================================

import os
import re
import html
import emoji
import unicodedata
import pickle
import logging
from datetime import datetime

import faiss
import pandas as pd
import numpy as np
from langdetect import detect, DetectorFactory
from indic_transliteration.sanscript import transliterate
from sentence_transformers import SentenceTransformer

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

DetectorFactory.seed = 0  # stable language detection

# ---- CONFIG ----
DB_INDEX = "hazard_vectors.index"
DB_META = "hazard_metadata.pkl"
DB_STATS = "database_stats.json"
INPUT_CSV = "to_save.csv"
LOG_CSV = "ingestion_log.csv"
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
EMBED_DIM = 384

# ---- Expanded Default Dataset ----
# Comprehensive mix of hazards + spam in all supported languages
DEFAULT_DATA = [
    # -------- Genuine Hazards (English) --------
    ("sea water is entering my shop near the beach", "genuine_hazard"),
    ("water is flooding houses near the coast", "genuine_hazard"),
    ("huge waves are hitting the boats in harbour", "genuine_hazard"),
    ("strong storm surge coming towards the village", "genuine_hazard"),
    ("cyclone wind is very strong near the seashore", "genuine_hazard"),
    ("road near the sea is completely flooded, cars stuck", "genuine_hazard"),
    ("bridge near the sea is partially submerged", "genuine_hazard"),
    ("sea level suddenly increased on the beach", "genuine_hazard"),

    # -------- Genuine Hazards (Hindi) --------
    ("समुद्र का पानी घरों के अंदर आ रहा है, तुरंत मदद भेजो", "genuine_hazard"),
    ("बहुत तेज लहरें आ रही हैं, समुद्र तट पर खड़ा होना खतरा है", "genuine_hazard"),
    ("तूफान की वजह से नाव डूब रही है, बचाव टीम भेजो", "genuine_hazard"),
    ("समुद्र किनारे की सड़क पूरी तरह से पानी में डूब गई है", "genuine_hazard"),

    # -------- Genuine Hazards (Marathi) --------
    ("समुद्राचं पाणी गल्लीमध्ये आलंय, मदत करा", "genuine_hazard"),
    ("इथे लाटा खूप मोठ्या येतायत, खूप धोका वाटतोय", "genuine_hazard"),
    ("बोट बुडत आहे, कृपया लगेच मदत पाठवा", "genuine_hazard"),
    ("समुद्राचं पाणी घरात शिरलंय, लवकर मदत हवी", "genuine_hazard"),

    # -------- Genuine Hazards (Hinglish / Roman Hindi-Marathi) --------
    ("pani ghar ke andar aa raha hai jaldi help chahiye", "genuine_hazard"),
    ("pani bharat ahe madat kara please", "genuine_hazard"),
    ("tez leher aa rahi hai beach band karna chahiye", "genuine_hazard"),
    ("samundar ka paani sadak pe aa gaya hai", "genuine_hazard"),
    ("samudra me paani badh raha hai, road block ho gaya", "genuine_hazard"),
    ("boat doob rahi hai, jaldi rescue bhejo", "genuine_hazard"),

    # -------- Spam / Non-hazard (English / Hinglish) --------
    ("just chilling at the beach with friends 😎", "spam"),
    ("fun time at sea nothing to worry", "spam"),
    ("enjoying sunset and waves, beautiful view", "spam"),
    ("no danger, only party on the beach tonight", "spam"),
    ("win free iphone now click this link", "spam"),
    ("best offer free recharge, click to claim", "spam"),
    ("ye sirf ek test message hai, ignore karo", "spam"),
    ("ghar par movie dekh raha hoon, koi dikkat nahi", "spam"),


    ("water from the tide is entering small shops near the harbour", "genuine_hazard"),
("waves are crashing over the safety wall and flooding the footpath", "genuine_hazard"),
("storm winds are hitting our coastal village, houses shaking badly", "genuine_hazard"),
("boats tied at the dock are drifting due to rough sea conditions", "genuine_hazard"),
("sea water is entering the fish market area, people evacuating", "genuine_hazard"),
("strong currents are pulling boats away from the shoreline", "genuine_hazard"),
("harbour road is underwater and vehicles are unable to cross", "genuine_hazard"),
("huge waves are pushing debris onto the beachside road", "genuine_hazard"),
("cyclone winds damaged several huts near the lighthouse", "genuine_hazard"),
("ocean water entered residential lanes, children stuck inside", "genuine_hazard"),
("tide levels suddenly increased and flooded the coconut farm", "genuine_hazard"),
("waves have reached the temple steps near the coast", "genuine_hazard"),
("storm surge collapsed small wooden stalls near the jetty", "genuine_hazard"),
("flooding reported near the coastal bus stand, water rising fast", "genuine_hazard"),
("sea water entered the parking area near the beach resort", "genuine_hazard"),
("fishermen stranded at sea due to high waves, need urgent help", "genuine_hazard"),
("sharp winds are throwing sand and debris on the coastal road", "genuine_hazard"),
("waves breached the first line of houses near the seafront", "genuine_hazard"),
("coastal electric poles fell due to intense wind pressure", "genuine_hazard"),
("boats anchored near the rocks are at risk of capsizing", "genuine_hazard"),
("storm tide is pushing water into the nearby agriculture fields", "genuine_hazard"),
("rough sea has washed away several fishing nets and equipment", "genuine_hazard"),
("sea water mixing with drain water causing overflow in lanes", "genuine_hazard"),
("ocean waves are hitting the compound walls behind the beach", "genuine_hazard"),
("harbour platform flooded making movement difficult", "genuine_hazard"),
("waves rising dangerously near the boat repair sheds", "genuine_hazard"),
("storm surge caused cracks in the coastal bridge pathway", "genuine_hazard"),
("strong tides pulled a small boat offshore unintentionally", "genuine_hazard"),
("sea level increased suddenly after evening tides", "genuine_hazard"),
("coastal farms are filling with salt water from the ocean", "genuine_hazard"),
("storm caused parts of the jetty to collapse", "genuine_hazard"),
("beach stalls destroyed due to continuous wave impact", "genuine_hazard"),
("waves entering the children’s play area next to the beach", "genuine_hazard"),
("lifeguard tower area partially submerged due to rising tides", "genuine_hazard"),
("coast guard issued a warning as waves reached residential zone", "genuine_hazard"),
("seawater blocked the pathways near the village well", "genuine_hazard"),
("beachside compound wall collapsed due to heavy wave pressure", "genuine_hazard"),
("waves are pushing boats onto the shore uncontrollably", "genuine_hazard"),
("seawater entered the local clinic near the coastal road", "genuine_hazard"),
("fishermen unable to return because of dangerous high waves", "genuine_hazard"),

("समुद्री लहरें लगातार बढ़ रही हैं, घरों में पानी घुसने लगा है", "genuine_hazard"),
("तूफानी हवाओं की वजह से कई झोपड़ियां उड़ गई हैं", "genuine_hazard"),
("समुद्र के पास का मैदान पूरी तरह पानी में डूब चुका है", "genuine_hazard"),
("लहरों की ऊंचाई बढ़ने से नावें टकरा रही हैं", "genuine_hazard"),
("समुद्री तूफान के चलते खेतों में पानी भर गया है", "genuine_hazard"),
("तेज हवा से समुद्र किनारे का बिजली का खंभा गिर गया", "genuine_hazard"),
("समुद्र के पानी ने स्कूल के मैदान को ढक लिया है", "genuine_hazard"),
("लहरें इतनी तेज हैं कि सड़कों पर चलना मुश्किल है", "genuine_hazard"),
("समुद्र तट पर खड़ी गाड़ियां पानी में फंस गई हैं", "genuine_hazard"),
("तूफान की वजह से छतें उड़ रही हैं और दीवारें गिर रही हैं", "genuine_hazard"),
("समुद्र का पानी बाज़ार की दुकानों में जाने लगा है", "genuine_hazard"),
("कई मछुआरे तेज लहरों में फंस गए हैं", "genuine_hazard"),
("लहरें लगातार सुरक्षा दीवार को तोड़ रही हैं", "genuine_hazard"),
("तूफानी लहरें रिहायशी इलाके तक पहुंच चुकी हैं", "genuine_hazard"),
("समुद्र का बहाव इतना तेज है कि लोग बहने लगे", "genuine_hazard"),

("समुद्राचं पाणी पायरीपर्यंत आलंय, लोक घाबरलेत", "genuine_hazard"),
("वारा इतका वेगाने सुटला की खिडक्या फुटल्या", "genuine_hazard"),
("लाटा सतत वाढतायत आणि रस्ता गायब झालाय", "genuine_hazard"),
("समुद्राचं पाणी दुकानांत भरत चाललंय", "genuine_hazard"),
("मच्छीमार समुद्रात अडकलेत आणि संपर्क तुटलाय", "genuine_hazard"),
("समुद्रकिनारीची बाग पूर्णपणे पाण्यात गेलीय", "genuine_hazard"),
("लाटा धक्कादायक वेगाने भिंतीवर आपटतायत", "genuine_hazard"),
("तुफानामुळे पाण्याचा वेग अनियंत्रित झालाय", "genuine_hazard"),
("समुद्राचं पाणी देवळाच्या परिसरातसुद्धा घुसलंय", "genuine_hazard"),
("लाटा इतक्या मोठ्या आहेत की घाट ओलांडतायत", "genuine_hazard"),

("samundar ka paani colony me aa chuka hai please help", "genuine_hazard"),
("heavy waves aa rahi hai beach ke pass jana unsafe hai", "genuine_hazard"),
("storm water ne pura ground bhar diya h", "genuine_hazard"),
("boat control me nahi h, leher bohot tez hai", "genuine_hazard"),
("samne ka pura area flood ho gaya h samundar ke paani se", "genuine_hazard"),
("coastal road pe paani chadh gaya h vehicles phase hue hain", "genuine_hazard"),
("waves ne stall udaa diya beach ke corner wala", "genuine_hazard"),
("samundar ka level abnormal lag raha h, log darr rahe", "genuine_hazard"),

("beach pe aaj mast vibe hai sab log enjoy kar rahe", "spam"),
("sunset dekhne ka mood tha, bohot accha view hai", "spam"),
("friends ke sath beachside coffee enjoy kar rahe", "spam"),
("mera new vlog aaj beach se record hua, check karo", "spam"),
("party chal rahi hai beach resort me, no tension", "spam"),
("free reward claim karo jaldi warna expire ho jayega", "spam"),
("aaj sirf fun beach trip hai koi danger nahi", "spam"),
("instagram reel banayi waves ke sath", "spam"),
("online contest join karo aur free voucher jeeto", "spam"),
("idhar sab safe aur cool atmosphere hai", "spam"),
("movie night chal rahi hai ghar par koi issue nahi", "spam"),
("chill session at the beach with squad", "spam"),
("koi dikkat nahi, sirf relaxing time", "spam"),
("giveaway chal raha insta pe participate karo", "spam"),
("friends ke sath masti chal rahi hai beach par", "spam"),
("random message hai ignore kar dena", "spam"),
("scam link mat kholna warna hack ho jayega", "spam"),
("beach par photo shoot ho raha hai, bohot fun", "spam"),
("just enjoying the cold breeze by the shore", "spam"),
("naya phone jeetne ka chance bas link click karo", "spam"),
("ye normal msg hai bas try kar raha hoon", "spam"),
("waves ke sath slow motion reel shoot ki", "spam"),
("food festival chal raha beach ke pass", "spam"),
("sab log chill kar rahe h yaha, no problems", "spam"),
("koi hazardous cheez nahi bas vibe hai", "spam"),
("new beach outfit try kar raha hoon", "spam"),
("nothing serious, just capturing moments", "spam"),
("free recharge loot sale chal rahi hai", "spam"),
("gaming session beach ke pass kar rahe", "spam"),
("idhar sab perfect weather enjoy time", "spam"),

("coastal market area is flooded after sudden high tide", "genuine_hazard"),
("storm waves are hitting the parked bikes near the sea wall", "genuine_hazard"),
("boats anchored offshore are struggling against rough currents", "genuine_hazard"),
("loud winds coming from the sea are shaking rooftops", "genuine_hazard"),
("sea water entered a small restaurant near the beach", "genuine_hazard"),
("waves pushed fishing crates far from the shore", "genuine_hazard"),
("a section of the coastal footpath has been washed away", "genuine_hazard"),
("large logs and debris washed in by strong waves", "genuine_hazard"),
("a fishing boat overturned due to unpredictable currents", "genuine_hazard"),
("coastal boundary wall cracked because of water pressure", "genuine_hazard"),
("harbour storage area is filling with seawater", "genuine_hazard"),
("waves are splashing onto the main road near the ocean", "genuine_hazard"),
("strong winds damaged the tin roofs near the shore", "genuine_hazard"),
("water reaching the fuel station near the beach road", "genuine_hazard"),
("tourists stuck near the rocks as waves rise higher", "genuine_hazard"),
("ocean water entered the backyard of seafront houses", "genuine_hazard"),
("storm clouds approaching fast, villagers moving inland", "genuine_hazard"),
("boats drifting towards the rocky edge of the harbour", "genuine_hazard"),
("street dogs trapped due to sudden sea flooding", "genuine_hazard"),
("sea water rising dangerously close to electric transformer", "genuine_hazard"),
("waves crossing warning flags on the beach", "genuine_hazard"),
("heavy tide damaged the wooden railings near the jetty", "genuine_hazard"),
("storm surge pushing water into the old church ground", "genuine_hazard"),
("boats colliding due to unstable currents", "genuine_hazard"),
("cyclone winds blowing away fishing nets and tools", "genuine_hazard"),
("coastal cemetery is partly submerged due to high tide", "genuine_hazard"),
("seawater leaking into underground storage near the port", "genuine_hazard"),
("waves washed away temporary sheds set up on the shore", "genuine_hazard"),
("big tides making it unsafe for people near the cliff area", "genuine_hazard"),
("fishermen signalling for help from a distance", "genuine_hazard"),
("ocean water moving inside coconut plantation areas", "genuine_hazard"),
("children stuck at the beach entrance due to waterlogging", "genuine_hazard"),
("tide entering the old railway track near the coastline", "genuine_hazard"),
("boats unable to dock because waves are too strong", "genuine_hazard"),
("stormwater entered the animal shelter near the beach", "genuine_hazard"),
("high waves shaking lamp posts near the promenade", "genuine_hazard"),
("harbour steps slippery and underwater due to tide", "genuine_hazard"),
("coastal farmland destroyed because of saltwater intrusion", "genuine_hazard"),
("seawater pushing trash bins into the main road", "genuine_hazard"),
("local fishermen shouting warnings about strong waves", "genuine_hazard"),
("rising waves forcing shops to shut early", "genuine_hazard"),

("समुद्र की लहरें अब खिडकियों तक पहुंच गई हैं", "genuine_hazard"),
("तूफानी हवाओं से दीवारें हिल रही हैं", "genuine_hazard"),
("समुद्र किनारे खड़ी नावें एक-दूसरे से टकरा रही हैं", "genuine_hazard"),
("पानी इतना बढ़ गया है कि लोगों को घर छोड़ना पड़ रहा है", "genuine_hazard"),
("समुद्र का बहाव खेतों को नुकसान पहुंचा रहा है", "genuine_hazard"),
("लहरें अचानक तेज हो गईं और सड़क डूबने लगी", "genuine_hazard"),
("तूफान के कारण बाज़ार का हिस्सा टूट गया", "genuine_hazard"),
("समुद्र के पास का पूरा इलाका अंधेरे में डूबा है", "genuine_hazard"),
("लहरों ने छोटे रेस्टोरेंट में पानी भर दिया है", "genuine_hazard"),
("लोग सुरक्षित जगह की ओर भाग रहे हैं", "genuine_hazard"),
("समुद्री पानी कुएं में घुस गया है", "genuine_hazard"),
("तूफानी लहरें झोपड़ियों को बहा ले गईं", "genuine_hazard"),
("समुद्र के पास की बिजली लाइन गिर चुकी है", "genuine_hazard"),
("समुद्री बहाव पुल पर चढ़ रहा है", "genuine_hazard"),
("लहरों की आवाज़ बहुत तेज हो चुकी है", "genuine_hazard"),

("समुद्राचं पाणी चौकात साचायला लागलंय", "genuine_hazard"),
("वारा घरांच्या पत्र्यांना उडवून नेत आहे", "genuine_hazard"),
("लाटा जेट्टीपर्यंत धडक देतायत", "genuine_hazard"),
("समुद्राचं पाणी राहत्या खोलीत सुद्धा आलंय", "genuine_hazard"),
("तूफानाच्या वाऱ्यामुळे झाडं एकदम कोसळली", "genuine_hazard"),
("लाटा इतक्या जोरात आहेत की बोट मागे ढकलली गेलीय", "genuine_hazard"),
("समुद्रकिनारीचा रस्ता पाण्याखाली गेला आहे", "genuine_hazard"),
("समुद्राचं मीठ पिकांना जाळतंय", "genuine_hazard"),
("गावात पाण्याचा आवाज सतत वाढतोय", "genuine_hazard"),
("किनाऱ्यावरची बुरूज सारखी हलतेय", "genuine_hazard"),

("samundar ka paani hostel ke andar aa raha h", "genuine_hazard"),
("waves bohot strong ho gayi h, log wapas ja rahe", "genuine_hazard"),
("fishermen ka group jetty me phase gaya hai", "genuine_hazard"),
("coastal road puri tarah se blocked h", "genuine_hazard"),
("storm water ka pressure bohot dangerous ho raha", "genuine_hazard"),
("beach ke peeche waala pura area flood ho chuka", "genuine_hazard"),
("samundar side shops band kar di gayi h", "genuine_hazard"),
("tide ne pura parking area bhar diya h", "genuine_hazard"),

("enjoying cold drinks at the beachside café", "spam"),
("nothing serious yaha, sab chill kar rahe", "spam"),
("new selfie clicked near the shore", "spam"),
("fun picnic going on with friends near the sea", "spam"),
("sunset ka reel record kiya, bohot accha laga", "spam"),
("online sale me free recharge mil raha, try karo", "spam"),
("waves ke sath photoshoot ho raha hai idhar", "spam"),
("sirf timepass karne aaye hai beach pe", "spam"),
("party scene active hai resort me", "spam"),
("giveaway join karo telegram pe", "spam"),
("beach walk kar raha hoon aaj weather perfect hai", "spam"),
("random msg test kar raha hoon", "spam"),
("no tension, bas holiday mood", "spam"),
("aaj ka vlogging beach tour upload kar diya", "spam"),
("sab friends enjoy kar rahe h near the shore", "spam"),
("scam messages aa rahe spam lag rahe", "spam"),
("idhar koi problem nahi everything normal", "spam"),
("beach par lighting bohot acchi lag rahi", "spam"),
("checking new camera near the rocks", "spam"),
("yaha sab kuch peaceful hai", "spam"),
("food stall try kiya beach ke pass", "spam"),
("waves ke sath slowmo shot banaya", "spam"),
("new music video shoot ho raha idhar", "spam"),
("students picnic celebrate kar rahe", "spam"),
("group chatting and chilling on beach", "spam"),
("online coupon mila claim kar liya", "spam"),
("nothing dangerous just enjoying weekend", "spam"),
("fresh air enjoy kar raha, no worries", "spam"),
("daily walk on beach good feeling", "spam"),
("sirf ek simple msg hai ignore kar do", "spam"),

("waves are hitting the shop shutters near the seafront", "genuine_hazard"),
("high tide has entered the coastal playground area", "genuine_hazard"),
("strong winds are bending trees dangerously near the shore", "genuine_hazard"),
("seawater is flowing across the small bridge near the fishing port", "genuine_hazard"),
("boats shaking heavily due to unpredictable sea movement", "genuine_hazard"),
("harbour maintenance area is partially underwater", "genuine_hazard"),
("waves have washed sand onto the main road blocking vehicles", "genuine_hazard"),
("rough waves are splashing into the bus stand near the beach", "genuine_hazard"),
("storm clouds forming rapidly over the ocean", "genuine_hazard"),
("snapped electric cables lying in water near the coast", "genuine_hazard"),
("sea water entering local hospital backyard near the shoreline", "genuine_hazard"),
("large tides pulling boats close to the rocky patch", "genuine_hazard"),
("waves breaking wooden platforms beside the harbour steps", "genuine_hazard"),
("coastal traffic stopped due to sudden flooding", "genuine_hazard"),
("security tents near the beach got damaged by high winds", "genuine_hazard"),
("seawater rising near the old fort wall beside the coast", "genuine_hazard"),
("ocean foam covering the entire walkway near the sea", "genuine_hazard"),
("early morning tide reached houses near the shoreline", "genuine_hazard"),
("dangerous rip current observed near the tourist zone", "genuine_hazard"),
("storm has broken several fishing lamps near the jetty", "genuine_hazard"),
("seawater flow carrying plastic drums into the village", "genuine_hazard"),
("waves louder than usual, indicating incoming storm", "genuine_hazard"),
("coastal traffic police requested people to move back", "genuine_hazard"),
("boats tied at the pier are lifting dangerously due to swell", "genuine_hazard"),
("sea water has reached the steps leading to the market", "genuine_hazard"),
("children stuck near the seafront school due to flooding", "genuine_hazard"),
("coastal village reported rising water levels within minutes", "genuine_hazard"),
("huge waves throwing fishing baskets all over the shore", "genuine_hazard"),
("storm surge pushing ocean water deep into the lanes", "genuine_hazard"),
("waves damaging roofs of several beachside cabins", "genuine_hazard"),
("sea water leaking through the back wall of houses near coast", "genuine_hazard"),
("boats drifting towards the marsh area behind the harbour", "genuine_hazard"),
("stormwater overflowing into the drainage system", "genuine_hazard"),
("seawater reaching the entrance of the local library", "genuine_hazard"),
("waves pushed big rocks closer to the living area near the sea", "genuine_hazard"),
("coastal boundary marked unsafe due to rising tides", "genuine_hazard"),
("rough tides shaking the old lighthouse structure", "genuine_hazard"),
("sea water entering private parking lots near seafront villas", "genuine_hazard"),
("ocean waves climbing over the promenade stairs", "genuine_hazard"),
("strong tides washing away fishing waste bins", "genuine_hazard"),

("समुद्र की अचानक उठी ऊंची लहरों से लोग घबरा गए हैं", "genuine_hazard"),
("तूफानी लहरें लगातार किनारे की तरफ बढ़ रही हैं", "genuine_hazard"),
("पानी भर जाने से गली का रास्ता बंद हो गया है", "genuine_hazard"),
("समुद्री हवा इतनी तेज है कि खिड़कियां हिल रही हैं", "genuine_hazard"),
("लहरों की वजह से छोटी नाव फंस गई है", "genuine_hazard"),
("समुद्र के पानी से खेत पूरी तरह डूब चुके हैं", "genuine_hazard"),
("तूफान के कारण बिजली का खंभा समुद्र में गिर गया है", "genuine_hazard"),
("लहरें दुकानों के भीतर तक पहुंच चुकी हैं", "genuine_hazard"),
("समुद्र तट की दीवार टूटने लगी है", "genuine_hazard"),
("पानी इतना बढ़ गया है कि लोग घरों में फंस गए हैं", "genuine_hazard"),
("तेज लहरें ट्रकों को खींचने लगी हैं", "genuine_hazard"),
("समुद्र का पानी मंदिर के मुख्य द्वार तक पहुंच गया है", "genuine_hazard"),
("तट पर बने छोटे पुल के नीचे का भाग डूब गया है", "genuine_hazard"),
("लहरें इतनी तेज हैं कि किनारे पर खड़ा रहना मुश्किल है", "genuine_hazard"),
("तूफान की वजह से बस्ती में अंधेरा फैल गया है", "genuine_hazard"),

("समुद्राचं पाणी सतत वाढतंय आणि घरे भिजतायत", "genuine_hazard"),
("लाटा इतक्या मोठ्या आहेत की रस्त्यावर धडकतायत", "genuine_hazard"),
("वारा इतका जोरात आहे की बोट हलूच शकत नाही", "genuine_hazard"),
("समुद्राचं पाणी विहिरीत मिसळलंय", "genuine_hazard"),
("गावापर्यंत पाण्याचा प्रवाह पोहोचला आहे", "genuine_hazard"),
("लाटा सुरक्षा भिंतीवर सतत आपटतायत", "genuine_hazard"),
("तुफानात मागच्या अंगणात पाणी भरून गेलं", "genuine_hazard"),
("समुद्रकाठचा बाग पूर्णपणे वाहून गेलाय", "genuine_hazard"),
("वाऱ्याने दिव्याचे खांब कोसळलेत", "genuine_hazard"),
("समुद्राचं पाणी इतकं वाढलं की बैलशेती बुडाली", "genuine_hazard"),

("samundar ka level itna badh gaya h ki road gayab lag raha", "genuine_hazard"),
("coastal area me water logging ho rahi h, log phase hue", "genuine_hazard"),
("tide ne puri boundary tod di beach ke pass wali", "genuine_hazard"),
("waves bohot heavy aa rahi, fishermen danger me", "genuine_hazard"),
("storm ne pura backyard bhar diya h samundar ke paani se", "genuine_hazard"),
("boat ko control karna mushkil ho raha unstable current me", "genuine_hazard"),
("samundar ke aas paas sirens baj rahe high tide alert", "genuine_hazard"),
("beach ke peeche wala pura block flood ho chuka h", "genuine_hazard"),

("music sunte hue beach walk kar raha hoon", "spam"),
("no danger here, just enjoying the cool wind", "spam"),
("reel banayi waves ke sath bohot acchi bani", "spam"),
("giveaway event live hai join karo", "spam"),
("friends ke sath snacks enjoy kar rahe", "spam"),
("waves ke sath photo click kar raha", "spam"),
("night walk on beach, peaceful time", "spam"),
("sirf timepass kar raha idhar", "spam"),
("new vlog upload hua beach tour ka", "spam"),
("nothing serious happening just fun", "spam"),
("nice ambience beachside restaurant ka", "spam"),
("sab safe lag raha koi dikkat nahi", "spam"),
("school picnic chal rahi near the shore", "spam"),
("contest join karo aur rewards jeeto", "spam"),
("scam messages aa rahe hain ignore karo", "spam"),
("fresh breeze enjoy kar raha, no tension", "spam"),
("waves ke sound ke sath relaxation time", "spam"),
("random test message bhej raha hoon", "spam"),
("click here to get free coupons", "spam"),
("group outing chal rahi beach par", "spam"),
("car parking near the sea full ho chuka", "spam"),
("photo session chal raha sunset ke time", "spam"),
("sab log enjoy kar rahe picnic location", "spam"),
("food tasting event beach pe lag raha", "spam"),
("bas vibe check karne aaya tha", "spam"),
("weather perfect tha isliye ghoomne aaya", "spam"),
("idhar koi storm nahi bas fun time", "spam"),
("friends ne new reel banayi idhar", "spam"),
("nothing urgent, chill mode on", "spam"),
("simple msg test kar raha hoon ignore", "spam"),
("strong waves are crashing into the fishermen’s huts near the coast", "genuine_hazard"),
("sea water has started entering the bicycle repair shop near the pier", "genuine_hazard"),
("tide pushed fishing boats far from their usual anchoring line", "genuine_hazard"),
("coastal security cabin is shaking due to heavy winds", "genuine_hazard"),
("waves are splashing over the boundary fence near the shore", "genuine_hazard"),
("market vendors are rushing to move their goods as sea water rises", "genuine_hazard"),
("water level rising behind the old fort near the sea", "genuine_hazard"),
("waves are covering the walking track along the coastline", "genuine_hazard"),
("storm surge entering the children’s park beside the beach", "genuine_hazard"),
("boats drifting unevenly as tide pulls them sideways", "genuine_hazard"),
("wheels of parked autos submerged due to seawater flow", "genuine_hazard"),
("waves breaking against temple entrance at the shoreline", "genuine_hazard"),
("storm winds knocking down the small stalls near the jetty", "genuine_hazard"),
("people trapped at the end of the boardwalk as waves block exit", "genuine_hazard"),
("saltwater filling the ground near the seafront apartments", "genuine_hazard"),
("tides rising close to the sewage treatment area", "genuine_hazard"),
("waves throwing small stones onto the coastal houses", "genuine_hazard"),
("harbour loading area unsafe due to slippery flooding", "genuine_hazard"),
("coastal road divider submerged completely", "genuine_hazard"),
("storm clouds bringing extremely rough waves toward the coast", "genuine_hazard"),
("fishermen reporting unstable currents far from the shoreline", "genuine_hazard"),
("sea water entering the small school next to the harbour", "genuine_hazard"),
("tide is lifting small fishing boats off the ground", "genuine_hazard"),
("sea waves causing cracks on the outer walls of beach shops", "genuine_hazard"),
("ocean water flowing through drainage channels towards the town", "genuine_hazard"),
("harbour warehouse floor flooded with seawater overnight", "genuine_hazard"),
("strong winds bending the signal poles near the promenade", "genuine_hazard"),
("wave height increasing rapidly, lifeguards issuing warnings", "genuine_hazard"),
("sea water reaching the entrance gate of community hall", "genuine_hazard"),
("storm surge causing local transport suspension", "genuine_hazard"),
("waves dragging plastic crates away from the fishing area", "genuine_hazard"),
("coastal road needs evacuation due to sudden waterlogging", "genuine_hazard"),
("seawater carried a broken boat piece into the alleyway", "genuine_hazard"),
("rip currents extremely strong near the rocky beach", "genuine_hazard"),
("waves lifting fishing boats dangerously high", "genuine_hazard"),
("seawater entering the first row of beachside houses", "genuine_hazard"),
("boat mechanics unable to work due to flooded workshop", "genuine_hazard"),
("waves hitting the staircase leading to the cliff viewpoint", "genuine_hazard"),
("stormwater pushing mud and debris onto the highway near the sea", "genuine_hazard"),

("समुद्र की तेज़ लहरें खिड़कियों तक पहुंच रही हैं", "genuine_hazard"),
("पानी घरों के आँगन में भरने लगा है", "genuine_hazard"),
("समुद्री तूफान के कारण स्कूल बंद कर दिया गया है", "genuine_hazard"),
("तेज बहाव से सड़क किनारे की दीवार गिर गई है", "genuine_hazard"),
("समुद्र का पानी खेतों में गहराई तक चला गया है", "genuine_hazard"),
("लहरों ने कई दुकानें डुबो दी हैं", "genuine_hazard"),
("तूफानी हवा से छत उड़ने लगी है", "genuine_hazard"),
("समुद्र के पास की बत्ती लगातार बुझ रही है", "genuine_hazard"),
("बस्ती में इतना पानी भर गया कि लोग बाहर नहीं निकल पा रहे", "genuine_hazard"),
("लहरें इतनी ऊंची हैं कि किनारा दिखाई नहीं दे रहा", "genuine_hazard"),
("समुद्री दूरी पर चलने वाले पुल के नीचे पानी चढ़ गया है", "genuine_hazard"),
("तबाही जैसी स्थिति बन रही है तट के पास", "genuine_hazard"),
("समुद्र के बहाव से पेड़ जड़ से उखड़ गए", "genuine_hazard"),
("लहरें घरों के दरवाजे तक आ चुकी हैं", "genuine_hazard"),
("तूफान के आने से पहले ही लोग सुरक्षित स्थान खोज रहे हैं", "genuine_hazard"),

("लाटा वाढल्यामुळे किनाऱ्यावरचा रस्ता गायब झालाय", "genuine_hazard"),
("समुद्राचं पाणी धानाच्या शेतात खोलवर गेलंय", "genuine_hazard"),
("मच्छीमारांनी परत यायला सुरूवात केली, समुद्र धोकादायक झालाय", "genuine_hazard"),
("लाटा भिंतीवर इतक्या जोरात आपटतायत की तडे पडलेत", "genuine_hazard"),
("तुफानामुळे वीजपुरवठा पूर्णपणे खंडित झालाय", "genuine_hazard"),
("समुद्रकाठचा घाट पाण्याखाली गेला आहे", "genuine_hazard"),
("लाटा इतक्या वेगाने येतायत की वस्तू वाहून जातायत", "genuine_hazard"),
("समुद्राचं पाणी मुख्य रस्त्यावर आलं आहे", "genuine_hazard"),
("वारा इतका आहे की छोटे झोपडे उलटलेत", "genuine_hazard"),
("समुद्राचं पाणी विहिरीपर्यंत पोहोचलंय", "genuine_hazard"),

("samundar ka paani main road tak aa gaya h", "genuine_hazard"),
("heavy tide aate hi pura chowk flood ho gaya", "genuine_hazard"),
("waves itni strong ho gayi ki log bhaag rahe", "genuine_hazard"),
("samudra side electricity trip ho gayi", "genuine_hazard"),
("fishermen ka group rescue ka wait kar raha h", "genuine_hazard"),
("storm ne beach ke peeche ki gali bhar di", "genuine_hazard"),
("waves ne ek choti boat ko palat diya", "genuine_hazard"),
("samundar ka flow bohot aggressive ho gaya h", "genuine_hazard"),

("friends ke sath bridge view enjoy kar rahe", "spam"),
("fun beach cycling chal rahi idhar", "spam"),
("photo kheech raha hoon waves ke sath", "spam"),
("idhar koi problem nahi sab thik hai", "spam"),
("giveaway ka result announce ho gaya", "spam"),
("evening snacks beachside stall se liye", "spam"),
("sunset ki lighting perfect lag rahi", "spam"),
("bas fun hangout kar rahe near seashore", "spam"),
("reel record karne aaye hai sab log", "spam"),
("light music sunke walk kar rahe", "spam"),
("shopping fest chal raha beach mall me", "spam"),
("random msg bheja test ke liye", "spam"),
("sab safe hai koi issue nahi", "spam"),
("free recharge ad aa raha bar bar", "spam"),
("group photo session chal raha", "spam"),
("fun picnic day with classmates", "spam"),
("waves ke pass bait kar relax kar rahe", "spam"),
("sunshine enjoy kar rahe beach par", "spam"),
("sirf casual outing nothing serious", "spam"),
("food stalls try kar rahe naye wale", "spam"),
("school friends ke sath fun time", "spam"),
("giveaway contest join kar liya", "spam"),
("romantic sunset enjoy kar rahe", "spam"),
("gaming stream beach ke pass kar raha", "spam"),
("new tshirt try kar li beach par", "spam"),
("nothing dangerous yaha all normal", "spam"),
("simple msg hai skip bhi kar sakte", "spam"),
("chill mode activated near seaface", "spam"),
("beautiful weather enjoy kar rahe", "spam"),
("bas random hi message bhej raha", "spam"),


]


# ==============================================================
# Enhanced Smart Cleaner with better preprocessing
# ==============================================================

class SmartCleaner:
    def __init__(self):
        self.url_re = re.compile(r"http\S+|www\.\S+")
        self.ws_re = re.compile(r"\s+")
        self.mention_re = re.compile(r"@\w+")
        self.hashtag_re = re.compile(r"#\w+")

    def translit_if_roman_hi_mr(self, text: str) -> str:
        """
        If text is detected as Hindi/Marathi & looks like roman letters,
        transliterate to Devanagari using ITRANS.
        """
        try:
            lang = detect(text)
        except:
            return text

        letters_only = text.replace(" ", "")
        if lang in ["hi", "mr"] and letters_only.isascii() and letters_only.isalpha():
            try:
                return transliterate(text, "itrans", "devanagari")
            except Exception as e:
                logger.debug(f"Transliteration failed: {e}")
                return text
        return text

    def clean(self, text: str) -> str:
        """Enhanced cleaning with better preprocessing"""
        if not isinstance(text, str):
            return ""

        # Basic cleanup
        text = html.unescape(text)
        text = unicodedata.normalize("NFKC", text)

        # Handle emojis - convert to text representation
        text = emoji.demojize(text, delimiters=(" ", " "))

        # Remove URLs, mentions, hashtags
        text = self.url_re.sub("", text)
        text = self.mention_re.sub("", text)
        text = self.hashtag_re.sub("", text)

        # Remove multiple spaces
        text = self.ws_re.sub(" ", text).strip()

        # Transliteration for Roman Hindi/Marathi
        text = self.translit_if_roman_hi_mr(text)

        return text.lower().strip()


# ==============================================================
# Enhanced Vector DB with validation and stats
# ==============================================================

class HazardVectorDB:
    def __init__(self):
        logger.info("🧠 Loading SentenceTransformer model...")
        self.model = SentenceTransformer(MODEL_NAME)
        self.cleaner = SmartCleaner()

        if os.path.exists(DB_INDEX) and os.path.exists(DB_META):
            logger.info("📌 Found existing FAISS DB, loading...")
            self.index = faiss.read_index(DB_INDEX)
            with open(DB_META, "rb") as f:
                self.metadata = pickle.load(f)
        else:
            logger.info("🆕 Creating new FAISS DB...")
            self.index = faiss.IndexFlatIP(EMBED_DIM)
            self.metadata = []

    def add_texts(self, texts, labels):
        """Add texts with deduplication and validation"""
        existing_clean = {m["clean"] for m in self.metadata}
        cleaner = self.cleaner

        new_records = []
        skipped = {"empty": 0, "duplicate": 0, "invalid": 0}

        for t, l in zip(texts, labels):
            # Validate label
            if l not in ["genuine_hazard", "spam", "uncertain"]:
                logger.warning(f"Invalid label '{l}' for text: {t[:50]}...")
                skipped["invalid"] += 1
                continue

            c = cleaner.clean(t)

            # Skip empty
            if not c:
                skipped["empty"] += 1
                continue

            # Skip duplicates
            if c in existing_clean:
                skipped["duplicate"] += 1
                continue

            new_records.append((t, c, l))
            existing_clean.add(c)

        if skipped["empty"] > 0:
            logger.info(f"⚠ Skipped {skipped['empty']} empty texts")
        if skipped["duplicate"] > 0:
            logger.info(f"⚠ Skipped {skipped['duplicate']} duplicate texts")
        if skipped["invalid"] > 0:
            logger.warning(f"⚠ Skipped {skipped['invalid']} texts with invalid labels")

        if not new_records:
            logger.warning("⚠ No new unique cleaned texts to add.")
            return 0

        cleaned_texts = [c for (_, c, _) in new_records]
        logger.info(f"🧠 Encoding {len(cleaned_texts)} new samples...")

        vectors = self.model.encode(
            cleaned_texts,
            normalize_embeddings=True,
            show_progress_bar=True
        ).astype("float32")

        self.index.add(vectors)

        for raw, clean, label in new_records:
            self.metadata.append({
                "raw": raw,
                "clean": clean,
                "label": label,
                "timestamp": datetime.now().isoformat()
            })

        logger.info(f"✅ Added {len(new_records)} vectors.")
        return len(new_records)

    def get_stats(self):
        """Calculate database statistics"""
        if not self.metadata:
            return {}

        labels = [m["label"] for m in self.metadata]
        from collections import Counter
        label_counts = Counter(labels)

        stats = {
            "total_vectors": self.index.ntotal,
            "label_distribution": dict(label_counts),
            "last_updated": datetime.now().isoformat(),
            "model": MODEL_NAME,
            "embedding_dim": EMBED_DIM
        }

        return stats

    def save(self):
        """Save index, metadata, and stats"""
        faiss.write_index(self.index, DB_INDEX)

        with open(DB_META, "wb") as f:
            pickle.dump(self.metadata, f)

        # Save statistics
        stats = self.get_stats()
        import json
        with open(DB_STATS, "w") as f:
            json.dump(stats, f, indent=2)

        logger.info(f"💾 Saved FAISS DB. Total vectors: {self.index.ntotal}")
        logger.info(f"📊 Label distribution: {stats['label_distribution']}")


# ==============================================================
# CSV + Ingestion Pipeline with validation
# ==============================================================

def ensure_csv():
    """Create default CSV if missing"""
    if not os.path.exists(INPUT_CSV) or os.path.getsize(INPUT_CSV) == 0:
        logger.info(f"⚠ {INPUT_CSV} missing/empty → creating default dataset")
        df = pd.DataFrame(DEFAULT_DATA, columns=["text", "label"])
        df.to_csv(INPUT_CSV, index=False, encoding="utf-8-sig")
        logger.info(f"✅ Created {INPUT_CSV} with {len(df)} rows")
    else:
        logger.info(f"📄 Found existing CSV: {INPUT_CSV}")

def validate_csv(df):
    """Validate CSV format and content"""
    errors = []

    if "text" not in df.columns:
        errors.append("Missing 'text' column")
    if "label" not in df.columns:
        errors.append("Missing 'label' column")

    if errors:
        return False, errors

    # Check for valid labels
    valid_labels = {"genuine_hazard", "spam", "uncertain"}
    invalid_labels = set(df["label"].unique()) - valid_labels

    if invalid_labels:
        errors.append(f"Invalid labels found: {invalid_labels}")
        errors.append(f"Valid labels are: {valid_labels}")

    # Check for empty texts
    empty_count = df["text"].isna().sum() + (df["text"] == "").sum()
    if empty_count > 0:
        logger.warning(f"Found {empty_count} empty text entries")

    return len(errors) == 0, errors

def run_ingestion():
    """Main ingestion pipeline with validation"""
    ensure_csv()

    try:
        df = pd.read_csv(INPUT_CSV, encoding="utf-8-sig")
    except Exception as e:
        logger.error(f"❌ Failed to read CSV: {e}")
        return

    # Validate CSV
    is_valid, errors = validate_csv(df)
    if not is_valid:
        logger.error("❌ CSV validation failed:")
        for error in errors:
            logger.error(f"  - {error}")
        return

    logger.info(f"✅ CSV validation passed. Processing {len(df)} rows...")

    texts = df["text"].astype(str).tolist()
    labels = df["label"].astype(str).tolist()

    db = HazardVectorDB()
    before = db.index.ntotal
    added = db.add_texts(texts, labels)
    db.save()
    after = db.index.ntotal

    if added > 0:
        # Save ingestion log
        log_df = pd.DataFrame(db.metadata[-added:])
        log_df.to_csv(LOG_CSV, index=False, encoding="utf-8-sig")
        logger.info(f"📄 Ingestion log saved to {LOG_CSV}")

    logger.info("="*70)
    logger.info(f"✨ Ingestion complete!")
    logger.info(f"   Vectors before: {before}")
    logger.info(f"   Vectors after: {after}")
    logger.info(f"   New vectors added: {added}")
    logger.info("="*70)


# ==============================================================
# Main Execution
# ==============================================================

if __name__ == "__main__":
    logger.info("🌊 BlueRadar Vector DB Creation Starting...")
    run_ingestion()
    logger.info("✅ Process completed successfully!")

2025-11-22 14:15:59,371 - INFO - 🌊 BlueRadar Vector DB Creation Starting...
2025-11-22 14:15:59,373 - INFO - 📄 Found existing CSV: to_save.csv
2025-11-22 14:15:59,399 - INFO - ✅ CSV validation passed. Processing 442 rows...
2025-11-22 14:15:59,400 - INFO - 🧠 Loading SentenceTransformer model...
2025-11-22 14:15:59,402 - INFO - Use pytorch device_name: mps
2025-11-22 14:15:59,402 - INFO - Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
2025-11-22 14:16:03,814 - INFO - 📌 Found existing FAISS DB, loading...
2025-11-22 14:16:04,891 - INFO - ⚠ Skipped 442 duplicate texts
2025-11-22 14:16:04,891 - WARNING - ⚠ No new unique cleaned texts to add.
2025-11-22 14:16:04,896 - INFO - 💾 Saved FAISS DB. Total vectors: 442
2025-11-22 14:16:04,896 - INFO - 📊 Label distribution: {'genuine_hazard': 314, 'spam': 128}
2025-11-22 14:16:04,896 - INFO - ======================================================================
2025-11-22 14:16:04,896 - INFO - ✨ Ing

In [ ]:
# ==============================================================
# BlueRadar – Search + Hazard Decision Layer
# Version 2.0 - Enhanced classification and confidence scoring
# ==============================================================

import os
import re
import html
import emoji
import unicodedata
import pickle
import json
import logging
from datetime import datetime
from typing import List, Dict, Any

import faiss
import numpy as np
from langdetect import detect, DetectorFactory
from indic_transliteration.sanscript import transliterate
from sentence_transformers import SentenceTransformer

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

DetectorFactory.seed = 0

# ---- CONFIG ----
DB_INDEX = "hazard_vectors.index"
DB_META = "hazard_metadata.pkl"
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
EMBED_DIM = 384

# ===============================================================
# ENHANCED KEYWORD LISTS - Comprehensive coverage
# ===============================================================

# Critical hazard keywords (higher weight)
CRITICAL_KEYWORDS = [
    # English
    "tsunami", "drowning", "sinking", "capsized", "emergency", "rescue",
    "urgent", "sos", "distress", "mayday",

    # Hindi
    "सुनामी", "डूब", "आपातकाल", "बचाव", "तुरंत", "आपात",

    # Marathi
    "त्सुनामी", "बुडत", "आपत्काल", "तातडीने", "बचाव",

    # Hinglis
    "doob", "dub", "bud"
]

# High-priority hazard keywords
HIGH_HAZARD_KEYWORDS = [
    # English
    "flood", "flooding", "flooded", "wave", "waves", "storm", "cyclone",
    "surge", "tide", "current", "erosion", "breach", "submerged",

    # Hindi
    "बाढ़", "पानी", "लहर", "लहरें", "तूफान", "चक्रवात", "समुद्र",
    "ज्वार", "भरती", "डूबा", "जलमग्न",

    # Marathi
    "पूर", "पाणी", "लाट", "लाटा", "वादळ", "चक्रीवादळ", "समुद्र",
    "भरती", "पाण्यात",

    # Hinglish
    "pani", "paani", "leher", "lehren", "tufan", "toofan",
    "samundar", "samudra"
]

# Medium-priority hazard keywords
MEDIUM_HAZARD_KEYWORDS = [
    # English
    "boat", "ship", "vessel", "harbor", "harbour", "port", "dock",
    "coast", "coastal", "shore", "beach", "sea", "ocean",

    # Hindi
    "नाव", "नौका", "जहाज़", "बंदरगाह", "किनारा", "तट", "समुद्र तट",

    # Marathi
    "नौका", "बोट", "जहाज", "बंदर", "किनारा", "समुद्रकिनार",

    # Hinglish
    "naav", "nauka", "boat", "kinara", "beach"
]

# Urgency/help keywords
URGENCY_KEYWORDS = [
    # English
    "help", "救命", "danger", "please", "now", "immediate", "critical",

    # Hindi
    "मदद", "सहायता", "खतरा", "कृपया", "अभी", "जल्दी",

    # Marathi
    "मदत", "साहाय्य", "धोका", "कृपया", "आता", "लवकर",

    # Hinglish
    "help", "madat", "madad", "jaldi", "jald", "bachao"
]

# Spam/irrelevant keywords
SPAM_KEYWORDS = [
    # English
    "click", "reward", "free", "iphone", "offer", "win", "discount",
    "recharge", "party", "fun", "travel package", "buy", "sale",
    "booking", "registration", "vacation", "photo", "selfie",
    "restaurant", "food", "delicious", "tasty", "yummy",

    # Hindi
    "मुफ्त", "क्लिक", "ऑफर", "पार्टी", "मज़ा", "मस्ती", "छूट",
    "रिचार्ज", "जीतें", "खरीदें", "बिक्री",

    # Marathi
    "मोफत", "क्लिक", "ऑफर", "पार्टी", "मजा", "मस्ती", "सूट",

    # Hinglish
    "free", "offer", "party", "maza", "masti", "discount", "mazaa"
]

# Negation words (reduce hazard score)
NEGATION_WORDS = [
    "not", "no", "never", "nothing", "नहीं", "नाही", "ना", "नही",
    "mat", "mत", "koi nahi"
]

# Weather/informational (context-dependent)
INFORMATIONAL_KEYWORDS = [
    "forecast", "report", "news", "update", "weather", "might",
    "possibly", "maybe", "heard", "saw earlier", "yesterday",
    "पूर्वानुमान", "समाचार", "अपडेट", "मौसम",
    "अंदाज", "बातमी", "हवामान"
]


# ==============================================================
# Smart Cleaner (same as DB creation)
# ==============================================================

class SmartCleaner:
    def __init__(self):
        self.url_re = re.compile(r"http\S+|www\.\S+")
        self.ws_re = re.compile(r"\s+")
        self.mention_re = re.compile(r"@\w+")
        self.hashtag_re = re.compile(r"#\w+")

    def translit_if_roman_hi_mr(self, text: str) -> str:
        try:
            lang = detect(text)
        except:
            return text

        letters_only = text.replace(" ", "")
        if lang in ["hi", "mr"] and letters_only.isascii() and letters_only.isalpha():
            try:
                return transliterate(text, "itrans", "devanagari")
            except Exception:
                return text
        return text

    def clean(self, text: str) -> str:
        if not isinstance(text, str):
            return ""

        text = html.unescape(text)
        text = unicodedata.normalize("NFKC", text)
        text = emoji.demojize(text, delimiters=(" ", " "))
        text = self.url_re.sub("", text)
        text = self.mention_re.sub("", text)
        text = self.hashtag_re.sub("", text)
        text = self.ws_re.sub(" ", text).strip()
        text = self.translit_if_roman_hi_mr(text)

        return text.lower().strip()


# ==============================================================
# Enhanced Keyword Intent Scoring
# ==============================================================

def keyword_intent_score(text: str) -> Dict[str, Any]:
    """
    Advanced keyword scoring with severity weighting and context analysis.

    Returns:
        Dict with score, breakdown, and flags
    """
    t = text.lower()
    score = 0
    breakdown = {
        "critical": 0,
        "high": 0,
        "medium": 0,
        "urgency": 0,
        "spam": 0,
        "negation": 0,
        "informational": 0
    }

    # Critical keywords (weight: 3)
    for kw in CRITICAL_KEYWORDS:
        if kw in t:
            breakdown["critical"] += 1
            score += 3

    # High-priority hazard keywords (weight: 2)
    for kw in HIGH_HAZARD_KEYWORDS:
        if kw in t:
            breakdown["high"] += 1
            score += 2

    # Medium-priority hazard keywords (weight: 1)
    for kw in MEDIUM_HAZARD_KEYWORDS:
        if kw in t:
            breakdown["medium"] += 1
            score += 1

    # Urgency keywords (weight: 1.5)
    for kw in URGENCY_KEYWORDS:
        if kw in t:
            breakdown["urgency"] += 1
            score += 1.5

    # Spam keywords (weight: -2)
    for kw in SPAM_KEYWORDS:
        if kw in t:
            breakdown["spam"] += 1
            score -= 2

    # Negation detection (multiply hazard score by 0.5)
    has_negation = any(neg in t for neg in NEGATION_WORDS)
    if has_negation:
        breakdown["negation"] = 1
        if score > 0:
            score *= 0.5

    # Informational context (weight: -0.5)
    for kw in INFORMATIONAL_KEYWORDS:
        if kw in t:
            breakdown["informational"] += 1
            score -= 0.5

    # Normalize to [-1, 1] range
    # Max possible positive score ~15, max negative ~10
    normalized_score = max(-1.0, min(1.0, score / 15.0))

    return {
        "score": round(normalized_score, 3),
        "raw_score": round(score, 2),
        "breakdown": breakdown,
        "has_negation": has_negation,
        "is_critical": breakdown["critical"] > 0,
        "is_urgent": breakdown["urgency"] > 0
    }


# ==============================================================
# Enhanced Hazard Search Engine
# ==============================================================

class HazardSearch:
    def __init__(self):
        if not os.path.exists(DB_INDEX) or not os.path.exists(DB_META):
            raise FileNotFoundError(
                "❌ Vector DB missing. Run blueradar_create_db.py first!"
            )

        logger.info("🔎 Loading search engine...")
        self.model = SentenceTransformer(MODEL_NAME)
        self.cleaner = SmartCleaner()
        self.index = faiss.read_index(DB_INDEX)

        with open(DB_META, "rb") as f:
            self.metadata = pickle.load(f)

        logger.info(f"✅ Loaded {self.index.ntotal} vectors from database")

    def _calculate_confidence(self, matches: List[Dict], kw_analysis: Dict) -> float:
        """
        Calculate confidence score based on:
        1. Top match similarity
        2. Consensus among top-k matches
        3. Keyword analysis alignment
        """
        if not matches:
            return 0.0

        top_score = matches[0]["score"]
        top_label = matches[0]["label"]

        # Check label consensus
        labels = [m["label"] for m in matches]
        consensus = labels.count(top_label) / len(labels)

        # Score variance (lower = more confident)
        scores = [m["score"] for m in matches]
        score_variance = np.var(scores) if len(scores) > 1 else 0

        # Keyword-vector alignment
        kw_score = kw_analysis["score"]
        kw_mapped = (kw_score + 1) / 2.0  # [-1,1] -> [0,1]

        if top_label == "genuine_hazard":
            alignment = kw_mapped
        elif top_label == "spam":
            alignment = 1 - kw_mapped
        else:
            alignment = 0.5

        # Weighted confidence calculation
        confidence = (
            0.50 * top_score +           # FAISS similarity
            0.25 * consensus +            # Label consensus
            0.15 * alignment +            # Keyword alignment
            0.10 * (1 - score_variance)   # Score consistency
        )

        return round(confidence, 3)

    def _make_decision(
        self,
        faiss_top_score: float,
        top_label: str,
        kw_analysis: Dict,
        confidence: float,
        matches: List[Dict]
    ) -> tuple:
        """
        Enhanced decision logic with confidence-based thresholds.

        Returns:
            (final_score, decision_text, priority_level)
        """
        kw_score = kw_analysis["score"]
        is_critical = kw_analysis["is_critical"]
        is_urgent = kw_analysis["is_urgent"]

        # Map keyword score to [0,1]
        kw_mapped = (kw_score + 1) / 2.0

        # Combined score
        final_score = 0.7 * faiss_top_score + 0.3 * kw_mapped

        # === CRITICAL ALERT ===
        if is_critical or (is_urgent and faiss_top_score > 0.6):
            return (
                final_score,
                "🚨 CRITICAL EMERGENCY - Immediate action required",
                "P0_CRITICAL"
            )

        # === LOW SIMILARITY (Unknown pattern) ===
        if faiss_top_score < 0.35:
            if kw_score > 0.5:
                return (
                    final_score,
                    "🆕 NEW HAZARD PATTERN (low DB match, high keywords) - Needs review",
                    "P1_HIGH"
                )
            else:
                return (
                    final_score,
                    "❓ UNKNOWN INPUT (low similarity, unclear intent)",
                    "P3_LOW"
                )

        # === GENUINE HAZARD CLASSIFICATION ===
        if top_label == "genuine_hazard":
            if confidence >= 0.85:
                return (
                    final_score,
                    "🚨 HIGH CONFIDENCE HAZARD - Dispatch response team",
                    "P0_CRITICAL"
                )
            elif confidence >= 0.70:
                return (
                    final_score,
                    "⚠️ PROBABLE HAZARD - Verify and prepare response",
                    "P1_HIGH"
                )
            elif confidence >= 0.55:
                return (
                    final_score,
                    "⚠️ POSSIBLE HAZARD - Monitor and investigate",
                    "P2_MEDIUM"
                )
            else:
                return (
                    final_score,
                    "🤔 UNCERTAIN HAZARD - Human review recommended",
                    "P2_MEDIUM"
                )

        # === SPAM CLASSIFICATION ===
        elif top_label == "spam":
            if confidence >= 0.80 and kw_score < 0:
                return (
                    final_score,
                    "🗑️ HIGH CONFIDENCE SPAM - Auto-filter",
                    "P4_IGNORE"
                )
            elif confidence >= 0.65:
                return (
                    final_score,
                    "❌ LIKELY SPAM - Low priority",
                    "P3_LOW"
                )
            else:
                return (
                    final_score,
                    "❔ POSSIBLY SPAM - Check for false negative",
                    "P3_LOW"
                )

        # === UNCERTAIN ===
        else:
            if kw_score > 0.3:
                return (
                    final_score,
                    "🤔 UNCERTAIN (hazard keywords present) - Human review needed",
                    "P2_MEDIUM"
                )
            else:
                return (
                    final_score,
                    "❔ UNCERTAIN - Cannot classify confidently",
                    "P3_LOW"
                )

    def search(self, queries: List[str], k: int = 5) -> List[Dict[str, Any]]:
        """
        Search and classify multiple queries.

        Args:
            queries: List of text queries to classify
            k: Number of nearest neighbors to retrieve

        Returns:
            List of classification results with detailed analysis
        """
        results = []

        for q in queries:
            # Clean the query
            cleaned_q = self.cleaner.clean(q)

            if not cleaned_q:
                results.append({
                    "query": q,
                    "cleaned": "",
                    "status": "INVALID",
                    "error": "Empty after cleaning",
                    "final_decision": "❌ INVALID INPUT",
                    "priority": "P4_IGNORE",
                    "confidence": 0.0
                })
                continue

            # Keyword analysis
            kw_analysis = keyword_intent_score(cleaned_q)

            # Vector search
            try:
                vec = self.model.encode(
                    [cleaned_q],
                    normalize_embeddings=True
                ).astype("float32")
                D, I = self.index.search(vec, k)
            except Exception as e:
                logger.error(f"Search error: {e}")
                results.append({
                    "query": q,
                    "cleaned": cleaned_q,
                    "status": "ERROR",
                    "error": str(e),
                    "final_decision": "❌ SEARCH ERROR",
                    "priority": "P3_LOW",
                    "confidence": 0.0
                })
                continue

            # Build matches list
            matches = []
            for score, idx in zip(D[0], I[0]):
                if idx < len(self.metadata):
                    meta = self.metadata[idx]
                    matches.append({
                        "score": float(round(score, 4)),
                        "label": meta["label"],
                        "raw": meta["raw"]
                    })

            # Calculate confidence
            confidence = self._calculate_confidence(matches, kw_analysis)

            # Make decision
            top_score = float(D[0][0])
            top_label = self.metadata[I[0][0]]["label"]

            final_score, decision, priority = self._make_decision(
                top_score,
                top_label,
                kw_analysis,
                confidence,
                matches
            )

            # Compile result
            results.append({
                "query": q,
                "cleaned": cleaned_q,
                "status": "SUCCESS",

                # Keyword analysis
                "keyword_analysis": {
                    "score": kw_analysis["score"],
                    "raw_score": kw_analysis["raw_score"],
                    "breakdown": kw_analysis["breakdown"],
                    "is_critical": kw_analysis["is_critical"],
                    "is_urgent": kw_analysis["is_urgent"],
                    "has_negation": kw_analysis["has_negation"]
                },

                # Vector search results
                "faiss_top_score": round(top_score, 3),
                "top_label": top_label,

                # Final classification
                "confidence": confidence,
                "final_score": round(final_score, 3),
                "final_decision": decision,
                "priority": priority,

                # Top matches
                "matches": matches,

                # Metadata
                "timestamp": datetime.now().isoformat()
            })

        return results

    def search_single(self, query: str, k: int = 5) -> Dict[str, Any]:
        """Convenience method for single query"""
        return self.search([query], k=k)[0]

    def batch_analyze(self, queries: List[str], k: int = 5) -> Dict[str, Any]:
        """
        Batch analysis with summary statistics.

        Returns:
            Dict containing results and aggregated stats
        """
        results = self.search(queries, k=k)

        # Calculate summary stats
        priorities = [r.get("priority", "UNKNOWN") for r in results]
        from collections import Counter
        priority_counts = Counter(priorities)

        decisions = [r.get("final_decision", "UNKNOWN") for r in results]

        summary = {
            "total_queries": len(queries),
            "successful": sum(1 for r in results if r.get("status") == "SUCCESS"),
            "failed": sum(1 for r in results if r.get("status") != "SUCCESS"),
            "priority_distribution": dict(priority_counts),
            "critical_alerts": sum(1 for r in results if r.get("priority") == "P0_CRITICAL"),
            "high_priority": sum(1 for r in results if r.get("priority") == "P1_HIGH"),
            "needs_review": sum(1 for r in results if "review" in r.get("final_decision", "").lower()),
            "avg_confidence": round(
                np.mean([r.get("confidence", 0) for r in results if r.get("status") == "SUCCESS"]),
                3
            ) if results else 0.0
        }

        return {
            "results": results,
            "summary": summary,
            "timestamp": datetime.now().isoformat()
        }


# ==============================================================
# Formatted Output Display
# ==============================================================

def print_result(result: Dict[str, Any], detailed: bool = True):
    """Pretty print a single result"""
    print("\n" + "="*80)
    print(f"📝 QUERY: {result['query']}")
    print(f"🧹 CLEANED: {result['cleaned']}")
    print("-"*80)

    if result["status"] != "SUCCESS":
        print(f"❌ STATUS: {result['status']}")
        print(f"   Error: {result.get('error', 'Unknown error')}")
        return

    # Priority and Decision
    priority_emoji = {
        "P0_CRITICAL": "🚨",
        "P1_HIGH": "⚠️",
        "P2_MEDIUM": "⚡",
        "P3_LOW": "ℹ️",
        "P4_IGNORE": "🗑️"
    }
    emoji_icon = priority_emoji.get(result["priority"], "❓")

    print(f"{emoji_icon} PRIORITY: {result['priority']}")
    print(f"🎯 DECISION: {result['final_decision']}")
    print(f"📊 CONFIDENCE: {result['confidence']:.1%}")
    print(f"🔢 FINAL SCORE: {result['final_score']:.3f}")

    if detailed:
        print("-"*80)
        # Keyword Analysis
        kw = result["keyword_analysis"]
        print(f"🔤 KEYWORD ANALYSIS:")
        print(f"   Score: {kw['score']:.3f} (raw: {kw['raw_score']:.2f})")
        print(f"   Critical: {kw['is_critical']} | Urgent: {kw['is_urgent']} | Negation: {kw['has_negation']}")
        print(f"   Breakdown: {kw['breakdown']}")

        # FAISS Results
        print(f"🔍 VECTOR SEARCH:")
        print(f"   Top Score: {result['faiss_top_score']:.3f}")
        print(f"   Top Label: {result['top_label']}")

        # Top Matches
        print(f"📚 TOP MATCHES:")
        for i, m in enumerate(result["matches"][:3], 1):
            print(f"   {i}. Score={m['score']:.3f} | Label={m['label']}")
            print(f"      Text: {m['raw'][:70]}...")

    print("="*80)


def print_batch_summary(analysis: Dict[str, Any]):
    """Print summary of batch analysis"""
    summary = analysis["summary"]

    print("\n" + "="*80)
    print("📊 BATCH ANALYSIS SUMMARY")
    print("="*80)
    print(f"Total Queries: {summary['total_queries']}")
    print(f"Successful: {summary['successful']}")
    print(f"Failed: {summary['failed']}")
    print(f"Average Confidence: {summary['avg_confidence']:.1%}")
    print("-"*80)
    print(f"🚨 Critical Alerts: {summary['critical_alerts']}")
    print(f"⚠️  High Priority: {summary['high_priority']}")
    print(f"🤔 Needs Review: {summary['needs_review']}")
    print("-"*80)
    print("Priority Distribution:")
    for priority, count in sorted(summary['priority_distribution'].items()):
        print(f"   {priority}: {count}")
    print("="*80)


# ==============================================================
# Example Usage & Testing
# ==============================================================

if __name__ == "__main__":
    logger.info("🌊 BlueRadar Search Engine Starting...")

    try:
        engine = HazardSearch()

        # Test queries covering various scenarios
        test_queries = [
            # Critical hazards
            "tsunami warning huge waves coming help",
            "पाणी भरत आहे मदत करा तुरंत",
            "boat डूब raha hai बचाव भेजो",

            # High priority
            "pani bhar raha hai please help jaldi",
            "समुद्र का पानी घर में आ गया है",
            "लाटा खूप मोठ्या येतायत धोका आहे",

            # Medium priority
            "waves looking bigger than usual today",
            "समुद्र थोड़ा उफान पर लग रहा है",

            # Spam
            "just chilling at beach bro fun time",
            "free reward click now win iphone",
            "बीच पर पार्टी हो रही है मज़ा आ रहा",

            # Edge cases
            "no danger everything is fine at coast",
            "heard there might be storm tomorrow",
            "कल रात शादी में आना मत, area me riot ho gaya. ",
            "கார்காலையில் இரசாயன சிந்தல் ஏற்பட்டுள்ளது, அருகில் வராதீர்கள்.",
            "sfsafsddvfv gewrgg  rgrgr ",
            "खूप जोरात वादळ वारा सुटला आहे किनारपट्टीवर आणि पाणी भरत आहे "
        ]

        logger.info(f"\n🧪 Testing with {len(test_queries)} queries...\n")

        # Batch analysis
        analysis = engine.batch_analyze(test_queries, k=5)

        # Print individual results
        for result in analysis["results"]:
            print_result(result, detailed=True)

        # Print summary
        print_batch_summary(analysis)

        # Save results to file
        output_file = "search_results.json"
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(analysis, f, indent=2, ensure_ascii=False)
        logger.info(f"\n💾 Results saved to {output_file}")

        logger.info("\n✅ Search engine test completed successfully!")

    except FileNotFoundError as e:
        logger.error(f"\n❌ {e}")
        logger.error("Please run blueradar_create_db.py first to create the vector database.")
    except Exception as e:
        logger.error(f"\n❌ Unexpected error: {e}")
        import traceback
        traceback.print_exc()

2025-11-22 17:58:29,809 - INFO - 🌊 BlueRadar Search Engine Starting...
2025-11-22 17:58:29,810 - INFO - 🔎 Loading search engine...
2025-11-22 17:58:29,813 - INFO - Use pytorch device_name: mps
2025-11-22 17:58:29,813 - INFO - Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
2025-11-22 17:58:34,554 - INFO - ✅ Loaded 442 vectors from database
2025-11-22 17:58:34,557 - INFO - 
🧪 Testing with 17 queries...



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-11-22 17:58:34,940 - INFO - 
💾 Results saved to search_results.json
2025-11-22 17:58:34,940 - INFO - 
✅ Search engine test completed successfully!



📝 QUERY: tsunami warning huge waves coming help
🧹 CLEANED: tsunami warning huge waves coming help
--------------------------------------------------------------------------------
🚨 PRIORITY: P0_CRITICAL
🎯 DECISION: 🚨 CRITICAL EMERGENCY - Immediate action required
📊 CONFIDENCE: 85.4%
🔢 FINAL SCORE: 0.780
--------------------------------------------------------------------------------
🔤 KEYWORD ANALYSIS:
   Score: 0.667 (raw: 10.00)
   Critical: True | Urgent: True | Negation: False
   Breakdown: {'critical': 1, 'high': 2, 'medium': 0, 'urgency': 2, 'spam': 0, 'negation': 0, 'informational': 0}
🔍 VECTOR SEARCH:
   Top Score: 0.757
   Top Label: genuine_hazard
📚 TOP MATCHES:
   1. Score=0.757 | Label=genuine_hazard
      Text: बहुत तेज लहरें आ रही हैं, समुद्र तट पर खड़ा होना खतरा है...
   2. Score=0.673 | Label=genuine_hazard
      Text: heavy waves aa rahi hai beach ke pass jana unsafe hai...
   3. Score=0.667 | Label=genuine_hazard
      Text: waves rising dangerously near the boat rep

In [ ]:
# Alternative approach - use transformers directly without sentence-transformers
import os
import sys

# Set environment variables
os.environ['TRANSFORMERS_NO_TF'] = '1'
os.environ['USE_TF'] = '0'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Core imports
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer
import torch.nn.functional as F

class SimpleSentenceTransformer:
    def __init__(self, model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.model.eval()

    def encode(self, texts, normalize_embeddings=True, show_progress_bar=False):
        if isinstance(texts, str):
            texts = [texts]

        embeddings = []
        for text in texts:
            inputs = self.tokenizer(text, return_tensors="pt", truncation=True,
                                  padding=True, max_length=512)

            with torch.no_grad():
                outputs = self.model(**inputs)
                # Mean pooling
                embedding = outputs.last_hidden_state.mean(dim=1).squeeze()

                if normalize_embeddings:
                    embedding = F.normalize(embedding, p=2, dim=0)

                embeddings.append(embedding.numpy())

        return np.array(embeddings).astype("float32")

# Test it
model = SimpleSentenceTransformer()
test_embedding = model.encode("This is a test sentence")
print(f"✅ Alternative model works! Embedding shape: {test_embedding.shape}")

✅ Alternative model works! Embedding shape: (1, 384)
